In [5]:
import queue
import math

In [6]:
# function to calculate the manhattan distance

def manhattanDistance(start, goal):
    return abs(start[0] - goal[0]) + abs(start[1] - goal[1])

In [7]:
class RepeatedAstar:
    
    def __init__(self, grid, start, goal):
        self.grid = grid
        self.start = start
        self.goal = goal
        self.known = [[manhattanDistance((i, j), goal) for i in range(len(grid[0]))] for j in range(len(grid))]
        
    def runAstar(self, current):
        self.known[self.start[0]][self.start[1]] = 0
        q = queue.PriorityQueue()
        q.put((0, self.start))
        while not q.empty():
            current = q.get()
            if current[1] == self.goal:
                return current[0]
            for i in self.getNeighbours(current[1]):
                if self.known[i[0]][i[1]] > current[0] + 1:
                    self.known[i[0]][i[1]] = current[0] + 1
                    q.put((current[0] + 1, i))
        return -1
    

In [8]:
a = [
    ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*'],
    ['*', 'S', '*', '*', '*', ' ', ' ', ' ', ' ', '*'],
    ['*', ' ', '*', '*', '*', ' ', '*', ' ', ' ', '*'],
    ['*', ' ', '*', '*', '*', ' ', '*', '*', ' ', '*'],
    ['*', ' ', '*', '*', ' ', ' ', '*', '*', ' ', '*'],
    ['*', ' ', '*', '*', ' ', ' ', '*', '*', ' ', '*'],
    ['*', ' ', ' ', ' ', ' ', '*', '*', '*', ' ', '*'],
    ['*', '*', '*', ' ', ' ', '*', '*', '*', 'T', '*'],
    ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
]
start = (1, 1)
goal = (7, 8)

known = [[manhattanDistance((i, j), goal) for i in range(len(a[0]))] for j in range(len(a))]

In [27]:

# Function to implement A* search algorithm


class aStar:
    def __init__(self, grid, known, start, goal):
        """
            Function that calculates the shortest path from start to goal given a grid and the known world
            known contains h value for each cell
            If a cell is an obstacle, the h value becomes infinity, 

            Args:
                grid (list): grid 
                known (list): known grid
                start (tuple): start coordinates
                goal (tuple): goal coordinates
        """    
        self.grid_dim = (len(grid[0]), len(grid))
        self.openList = queue.PriorityQueue()   
        self.closedList = []
        
        self.openList.put(start, start)
        # g Values
        gMatrix = [[0 for i in range(len(grid[0]))] for j in range(len(grid))]

        while self.openList:
            print("Open List is", self.openList.queue)
            current = self.openList.get()
            print("Agent is at", current)
            neighbours = self.getNeighbours(current)
            
            # Observe surroundings
            for i in range(len(neighbours)):
                print(neighbours[i])
                if neighbours[i] == goal:
                    print("Found goal")
                    #TODO: Return path
                    return
                
                # update obstacle values
                if grid[neighbours[i][0]][neighbours[i][1]] == "*":
                    known[neighbours[i][0]][neighbours[i][1]] = math.inf
                else:
                    known[neighbours[i][0]][neighbours[i][1]] = 0
                g = gMatrix[current[0]][current[1]] + 1
                h = known[neighbours[i][0]][neighbours[i][1]]
                f = g + h
                
                present, f_check = self.checkCellInOpenList(current)
                print("Checking open list")
                if present:
                    if f > f_check:
                        continue
                
                present, f_check = self.checkCellInClosedList(current)
                print("Checking closed list")
                if present:
                    if f > f_check:
                        continue
                        
                else:
                    self.openList.put(neighbours[i])
                    gMatrix[neighbours[i][0]][neighbours[i][1]] = g    
                
            self.closedList.append(current)


    def getNeighbours(self,pos):
        neighbours = []
        
        if pos[0] > 0:
            neighbours.append((pos[0] - 1, pos[1]))
        if pos[0] < self.grid_dim[0] - 1:
            neighbours.append((pos[0], pos[1] + 1))
        if pos[1] > 0:
            neighbours.append((pos[0], pos[1] - 1))
        if pos[1] < self.grid_dim[1] - 1:
            neighbours.append((pos[0] + 1, pos[1]))
        
        return neighbours
    
    def checkCellInOpenList(self, pos):
        for i in self.openList.queue:
            if i == pos[1]:
                return True, pos[0]
        return False, 0
    
    def checkCellInClosedList(self, pos):
        for i in self.closedList:
            if i[1] == pos:
                return True, i[0]
        return False, 0
    
    
    

In [28]:
test = aStar(a, known, start, goal)

Open List is [(1, 1)]
Agent is at (1, 1)
Checking open list
Checking closed list
Checking open list
Checking closed list
Checking open list
Checking closed list
Checking open list
Checking closed list
Open List is [1, inf, inf, inf]
Agent is at 1


TypeError: 'int' object is not subscriptable